# Adjoint sensitivity method for Kohn Sham equations (supplementary material)
### Evgeny M. Kadilenko (NSU), Roland Grinis (MIPT, GrinisRIT)
#### Assistants: Gregory Dushkin (MIPT), Sabina Abdiganieva (MIPT)

## Preparing the globals

### Imports

In [1]:
import dqc
from pyscf import gto, dft, scf, cc
import numpy
import torch
import xitorch as xt
from math import pi
import sys 
import time
import pandas as pd
from IPython.display import display, HTML, Math
import numba as nb
import matplotlib.pyplot as plt

### Molecules definitions

In [2]:
molecules = {
    "h2"       :  """
                  H 0 0 0; H 1.4 0 0
                  """
    ,
    "n2"       :  """
                  N 0 0 0; N 2.07 0 0
                  """
    ,
    "water"    : """
                 O        0.000000000      0.000000000      0.000000000;
                 H        0.000000000      1.434938863      1.126357947;
                 H        0.000000000     -1.434938863      1.12635794
                 """
     ,
     "ammonia" : """
                 N  0.000  0.000  0.000;
                 H  0.000 -1.772 -0.721; 
                 H  1.535  0.886 -0.721; 
                 H -1.535  0.886 -0.721
                 """
}

### Test results storage

In [3]:
results = {
    "pyscf":            {},
    "pyscf_custom":   {},
    "pyscf_ccsdt":      {},
    "pyscf_adjoint":    {},
    "dqc":              {},
    "dqc_autograd":     {}
}

for method in results:
    for mol in molecules:
        results[method][mol] = {}

## Custom functionals

Here we take a look at the ways to implement an LDA functional and compare them to some built-in methods.

### PySCF

PySCF accepts custom functionals as Python functions

In [4]:
def custom_eval_xc_LDA(a, p):
    def eval_xc(xc_code, rho, *args, **kwargs):
        exc = a * rho ** (p - 1.0)
        vrho = a * p * rho ** (p - 1.0)
        vxc = (vrho, None, None, None)
        fxc = a * p * (p - 1.0) * rho ** (p - 2.0)
        return exc, vxc, fxc, None
    return eval_xc

In [5]:
def scf_custom(mol_name, a = 1, p = 2, silent = False):
    start = time.time()
    mol = molecules[mol_name]
    scf_mol = gto.M(atom=mol, basis="6-31G", unit="Bohr", verbose=0)
    calc_scf = dft.RKS(scf_mol)
    calc_scf = calc_scf.define_xc_(custom_eval_xc_LDA(a, p), "LDA")
    energy = calc_scf.kernel()
    dur = time.time() - start
    if not silent:
        print(f"For {mol_name} {energy=} ({a=}, {p=})")
        if not calc_scf.converged:
            print(f"Calculation for {mol_name} did not converge", file=sys.stderr)
    return energy, calc_scf.converged, dur

#### Built-in functional
To compare the results, we would use PySCF's built-in implementation of the same functional

In [88]:
def scf_builtin(_mol_name, silent = False):
    start = time.time()

    mol = molecules[_mol_name]
    scf_mol = gto.M(atom=mol, basis="6-31G", unit="Bohr", verbose=0)
    calc_scf = dft.RKS(scf_mol)
    calc_scf.xc = "lda," # Built-in LDA functional implementation
    energy = calc_scf.kernel()

    dur = time.time() - start
    
    if not silent:
        print(f"For {_mol_name} {energy=}")
        if not calc_scf.converged:
            print(f"Calculation for {_mol_name} did not converge", file=sys.stderr)
    return energy, calc_scf.converged, dur

#### CCSD(T)

In [89]:
def scf_ccsdt(_mol_name, silent = False):
    start = time.time()

    mol = molecules[_mol_name]
    scf_mol = gto.M(atom=mol, basis="6-31G", unit="Bohr", verbose=0)
    hf = scf.HF(scf_mol).run()
    calc_scf = cc.CCSD(hf)
    _en = calc_scf.kernel()
    et = calc_scf.ccsd_t()
    energy = calc_scf.e_tot + et

    dur = time.time() - start
    
    if not silent:
        print(f"For {_mol_name} {energy=}")
        if not calc_scf.converged:
            print(f"Calculation for {_mol_name} did not converge", file=sys.stderr)
    return energy, calc_scf.converged, dur

### DQC

DQC accepts custom functionals as objects of classes derived from `dqc.xc.CustomXC`

In [90]:
class MyLDAX(dqc.xc.CustomXC):
    def __init__(self, a_par, p_par):
        super().__init__()
        self.a_par = a_par
        self.p_par = p_par
        self.number_of_parameters = 2

    @property
    def family(self):
        # 1 for LDA, 2 for GGA, 4 for MGGA
        return 1

    def get_edensityxc(self, densinfo):
        # densinfo has up and down components
        if isinstance(densinfo, dqc.utils.SpinParam):
            # spin-scaling of the exchange energy
            return 0.5 * (self.get_edensityxc(densinfo.u * 2) + self.get_edensityxc(densinfo.d * 2))
        else:
            rho = densinfo.value.abs() + 1e-15  # safeguarding from nan
            return self.a_par * rho ** self.p_par
        
    def get_edensityxc_derivative(self, densinfo, number_of_parameter):
        # densinfo has up and down components
        if isinstance(densinfo, dqc.utils.SpinParam):
            # spin-scaling of the exchange energy
            return 0.5 * (self.get_edensityxc_derivative(densinfo.u * 2, number_of_parameter) 
                          + self.get_edensityxc_derivative(densinfo.d * 2, number_of_parameter))
        else:
            rho = densinfo.value.abs() + 1e-15  # safeguarding from nan
            if number_of_parameter == 0: # parameter a
                return rho ** self.p_par
            elif number_of_parameter == 1: # parameter p
                return self.a_par * torch.log(rho) * rho ** self.p_par

In [91]:
def dqc_custom(_mol_name, a = 1.0, p = 2.0, silent = False):
    start = time.time()

    a_par = torch.nn.Parameter(torch.tensor(a, dtype=torch.double))
    p_par = torch.nn.Parameter(torch.tensor(p, dtype=torch.double))
    mol = molecules[_mol_name]
    atmozs, atomposs = dqc.parse_moldesc(mol)
    dqc_mol = dqc.Mol((atmozs, atomposs), basis="6-31G")
    qc = dqc.KS(dqc_mol, xc=MyLDAX(a_par, p_par)).run()
    energy_tensor = qc.energy()
    energy = energy_tensor.item()

    dur = time.time() - start

    if not silent:
        print(f"For {_mol_name} {energy=} ({a=}, {p=})")
    return energy_tensor, True, dur

### Comparison test

Let's compare all of the above methods (also, for convenience, all of the results are compiled at the end of the notebook in tables).

In [92]:
factor = -3.0/4.0 * (3.0 / pi)**(1.0/3.0)
power = 4.0/3.0

for mol in molecules:
    print("Builtin:       ", end='')
    e, c, d = scf_builtin(mol)
    results["pyscf"][mol]["energy"] = e
    results["pyscf"][mol]["converged"] = c
    results["pyscf"][mol]["time"] = d
    print("CCSD(T):       ", end='')
    e, c, d = scf_ccsdt(mol)
    results["pyscf_ccsdt"][mol]["energy"] = e
    results["pyscf_ccsdt"][mol]["converged"] = c
    results["pyscf_ccsdt"][mol]["time"] = d
    print("PySCF custom:  ", end='')
    e, c, d = scf_custom(mol, factor, power)
    results["pyscf_custom"][mol]["energy"] = e
    results["pyscf_custom"][mol]["converged"] = c
    results["pyscf_custom"][mol]["time"] = d
    print("DQC custom:    ", end='')
    e, c, d = dqc_custom(mol, factor, power)
    results["dqc"][mol]["energy"] = e.item()
    results["dqc"][mol]["converged"] = c
    results["dqc"][mol]["time"] = d
    print("PySCF custom:  ", end='')
    print("---><---")

Builtin:       For h2 energy=-1.0386177856738823
CCSD(T):       For h2 energy=-1.1516791660767651
PySCF custom:  For h2 energy=-1.0386177856738827 (a=-0.7385587663820223, p=1.3333333333333333)
DQC custom:    For h2 energy=-1.0386170699228434 (a=-0.7385587663820223, p=1.3333333333333333)
PySCF custom:  ---><---
Builtin:       For n2 energy=-107.63947353831068
CCSD(T):       For n2 energy=-109.10264680237505
PySCF custom:  For n2 energy=-107.63947353831057 (a=-0.7385587663820223, p=1.3333333333333333)
DQC custom:    For n2 energy=-107.6394731463358 (a=-0.7385587663820223, p=1.3333333333333333)
PySCF custom:  ---><---
Builtin:       For water energy=-75.1547053402564
CCSD(T):       For water energy=-76.1206412072075
PySCF custom:  For water energy=-75.15470534025634 (a=-0.7385587663820223, p=1.3333333333333333)
DQC custom:    For water energy=-75.15469465302141 (a=-0.7385587663820223, p=1.3333333333333333)
PySCF custom:  ---><---
Builtin:       For ammonia energy=-55.413604406782774
CCSD(

<ipython-input-86-ad976f8892da>:6: RuntimeWarning: divide by zero encountered in power
  fxc = a * p * (p - 1.0) * rho ** (p - 2.0)


## Calculating energy derivatives

Some basic ways to compute energy derivatives are finite differences. DQC also allows to make use of PyTorch's `autograd`.

From now on, all of the calculations will rely on user defined functionals.

### Finite differences

Calculating derivatives via finite difference is straighforward once there is a way to calculate energy:

In [93]:
def fd(method, _mol_name, a, p, bump_factor=0.00001):
    c = True
    en, c1, _ = method(_mol_name, a, p, silent = True)
    c &= c1
    da = a * bump_factor
    dp = p * bump_factor

    en_da, c1, _ = method(_mol_name, a + da, p, silent = True)
    c &= c1
    en_dp, c1, _ = method(_mol_name, a, p + dp, silent = True)
    c &= c1

    dea = (en_da - en) / da
    dep = (en_dp - en) / dp

    print(f"For {_mol_name}:")
    print(f"dE/da = {dea}")
    print(f"dE/dp = {dep}")

    if not c:
        print(f"One of finite difference computations for {_mol_name} didn't converge with {method}")

    return dea, dep

### DQC and `torch.autograd`

In [94]:
def dqc_autograd(_mol_name, a, p):
    # Mostly same as e_dqc()
    a_par = torch.nn.Parameter(torch.tensor(a, dtype=torch.double))
    p_par = torch.nn.Parameter(torch.tensor(p, dtype=torch.double))
    mol = molecules[_mol_name]
    atmozs, atomposs = dqc.parse_moldesc(mol)
    dqc_mol = dqc.Mol((atmozs, atomposs), basis="6-31G")
    qc = dqc.KS(dqc_mol, xc=MyLDAX(a_par, p_par)).run()
    energy_tensor = qc.energy()
    grad = torch.autograd.grad(energy_tensor, (a_par, p_par))

    print(f"For {_mol_name}:")
    print(f"dE/da = {grad[0]}")
    print(f"dE/dp = {grad[1]}")
    return grad

### Comparison test

Derivatives computations are compared at the end of the notebook.

## Adjoint derivatives

### Full algorithm in DQC

#### Preparing for calculation

Initialize the system, DQC, and set up some variables

In [95]:
from dqc.utils.datastruct import SpinParam, ValGrad

a = 1.0
p = 2.0
mol_name = "water"

a_par = torch.nn.Parameter(torch.tensor(a, dtype=torch.double))
p_par = torch.nn.Parameter(torch.tensor(p, dtype=torch.double))
aj_mol = molecules[mol_name]
atmozs, atomposs = dqc.parse_moldesc(aj_mol)
dqc_mol = dqc.Mol((atmozs, atomposs), basis="6-31G")
qc = dqc.KS(dqc_mol, xc=MyLDAX(a_par, p_par)).run()

In [96]:
def _symm(scp: torch.Tensor):
    # forcely symmetrize the tensor
    return (scp + scp.transpose(-2, -1)) * 0.5

dm = qc._dm.detach().clone()
noccorb = qc._engine.hf_engine._norb
nallorb = qc.get_system().get_hamiltonian().nao
fockian = qc._engine.dm2scp(dm).detach().clone()

def eigendecomposition(hf_engine):
    if not hf_engine._polarized:
        fock = xt.LinearOperator.m(_symm(fockian), is_hermitian=True)
        return hf_engine.diagonalize(fock, noccorb)
    else:
        fock_u = xt.LinearOperator.m(_symm(fockian[0]), is_hermitian=True)
        fock_d = xt.LinearOperator.m(_symm(fockian[1]), is_hermitian=True)
        return hf_engine.diagonalize(hf_engine, SpinParam(u=fock_u, d=fock_d))

energies, coefficients = eigendecomposition(qc._engine.hf_engine)
orboccs = qc._engine.orb_weight.detach().clone()

First, derivatives from energy:

#### Calculating $\frac{\partial E[\rho](attachment:./\vec{\theta})}{\partial \textbf{C}}$

Only canonical (i.e. eigenfunctions of fockian) orbitals can be used:
$$ \frac{\partial E[\rho](attachment:./\vec{\theta})}{\partial C_{bj}} = 2f_b \sum_i C_{bi}F_{ij} = 2f_b \epsilon_b C_{bj}$$
so
$$ \frac{\partial E[\rho](attachment:./\vec{\theta})}{\partial \textbf{C}} = 2\textbf{f}\epsilon\textbf{C}$$
$\epsilon$ here is matrix: $\epsilon_{ab}=\delta_{ab}\epsilon_a$

In [97]:
dE_wrt_dC = 2 * torch.einsum("b,ib,ij->bj", orboccs, coefficients, fockian)

#### Calculating $\frac{\partial E[\rho](attachment:./\vec{\theta})}{\partial \vec{\epsilon}}$

$$\frac{\partial E[\rho](attachment:./\vec{\theta})}{\partial \epsilon_{b}} = 0$$ for all $\epsilon_b$. So:

In [98]:
dE_wrt_depsilon = torch.zeros(noccorb)

Then, derivatives from normalization equations:

#### Calculating $\frac{\partial \vec{r}(\textbf{C})}{\partial \textbf{C}}$

$$\frac{\partial r_{a}(\textbf{C})}{\partial C_{ck}} = 2\delta_{ac}C_{ck}$$

In [99]:
occupied_orbitals_kronecker = torch.eye(noccorb)
dnorm_wrt_dC = 2 * torch.einsum("ac,kc->kac", occupied_orbitals_kronecker, coefficients)

#### Calculating $\frac{\partial \vec{r}(\textbf{C})}{\partial \vec{\epsilon}}$
$$\frac{\partial r_{a}(\textbf{C})}{\partial \epsilon_b} = 0$$

In [100]:
dnorm_wrt_depsilon = torch.zeros((noccorb, noccorb))

Derivatives from Roothan equations:

#### Calculating $\frac{\partial \textbf{r}(\textbf{C};\;\vec{\theta})}{\partial \textbf{C}}$

$$G_{Cou, ijkl} = \iint b_i(\vec{r}) \frac{b_k(\vec{r'})b_l(\vec{r'})}{|\vec{r}-\vec{r'}|} b_j(\vec{r})d\vec{r'}d\vec{r}$$
$$G_{XC, ijkl} = 
\int b_i(\vec{r}) b_k(\vec{r}) \frac{\partial V_{XC}[\rho](attachment:./\vec{r};\;\vec{\theta})}{\partial \rho(\vec{r})}b_l(\vec{r})b_j(\vec{r})d\vec{r}$$
$$\frac{\partial r_{ia}(\textbf{C};\;\vec{\theta})}{\partial C_{ck}} = 
(F_{ik}[\rho](attachment:./\vec{\theta}) - \epsilon_c\delta_{ik})\delta_{ac} +
2f_c\sum_j \sum_{l}C_{cl}\left(G_{Cou,ijkl} + G_{XC,ijkl}\right)C_{aj}
$$

First term:

In [101]:
all_orbitals_kronecker = torch.eye(nallorb)
dRoothan_wrt_dC_first_term = torch.einsum(
                                            "ik,ac->iakc",
                                            fockian,
                                            occupied_orbitals_kronecker
                                        )
dRoothan_wrt_dC_first_term -= torch.einsum(
                                            "c,ik,ac->iakc",
                                            energies,
                                            all_orbitals_kronecker,
                                            occupied_orbitals_kronecker
                                        )

Second term (fourDtensor loses symmetry of XC-tensor):

In [102]:
four_center_elrep_tensor = torch.einsum(
                "ijkl,lc,ja->iakc",
                qc._engine.hamilton.el_mat.detach().clone(),
                coefficients,
                coefficients
            )

def get_dvxc_wrt_dro_xc(xc, densinfo):
    # mark the densinfo components as requiring grads
    with xc._enable_grad_densinfo(densinfo):
        with torch.enable_grad():
            edensity = xc.get_edensityxc(densinfo)  # (*BD, ngrid)
        grad_outputs = torch.ones_like(edensity)
        # print(f'edensity {edensity.shape}')
        grad_enabled = torch.is_grad_enabled()

        if not isinstance(densinfo, ValGrad):  # polarized case
            raise NotImplementedError("polarized case is not implemented")
        else:  # unpolarized case
            if xc.family == 1:  # LDA
                # Here we are taking autograd.grad derivative twice:
                potinfo, = torch.autograd.grad(
                    edensity, densinfo.value, create_graph=grad_enabled,
                    grad_outputs=grad_outputs)
                # print(f'potinfo {potinfo.shape}')
                derivative_of_potinfo_wrt_ro, = torch.autograd.grad(
                    potinfo, densinfo.value, grad_outputs=grad_outputs)
                return ValGrad(value=derivative_of_potinfo_wrt_ro)
            else:  # GGA and others
                raise NotImplementedError("Default dvxc wrt dro for this family is not implemented")

@nb.njit(parallel=True)
def _four_term_integral(m, b, o):
    N = b.shape[1]  # size of basis set
    n = o.shape[1]  # number of occupied orbitals
    R = m.shape[0]  # size of grid
    res = numpy.zeros((N, n, N, n))
    for i in nb.prange(N):
        for a in range(n):
            for k in range(i+1):
                for c in range(a+1):
                    for r in range(R):
                        value = m[r] * b[r, i] * o[r, a] * b[r, k] * o[r, c]
                        res[i, a, k, c] += value
                        if a != c:
                            res[i, c, k, a] += value
                            if i != k:
                                res[k, a, i, c] += value
                                res[k, c, i, a] += value
                        elif i != k:
                            res[k, a, i, c] += value
    return res

def get_dvxc_wrt_dro_from_derivative_of_potinfo_wrt_ro(hamiltonian, derivative_of_potinfo_wrt_ro: ValGrad):
    print(f'started the four term integral computation for {hamiltonian.basis.shape}...')
    molecular_orbitals_at_grid = hamiltonian.basis @ coefficients
    derivative_of_potinfo_wrt_ro_dvolume = derivative_of_potinfo_wrt_ro.value * hamiltonian.dvolume
    mat = _four_term_integral(derivative_of_potinfo_wrt_ro_dvolume.numpy(),
                                hamiltonian.basis.numpy(),
                                molecular_orbitals_at_grid.numpy())
    return torch.from_numpy(mat)

def construct_vxc_derivative_tensor(hamiltonian, dm):
    densinfo = SpinParam.apply_fcn(lambda dm_: hamiltonian._dm2densinfo(dm_), dm)  # value: (*BD, ngrid)
    print(f'density info {densinfo.value.shape}')
    derivative_of_potinfo_wrt_ro = get_dvxc_wrt_dro_xc(hamiltonian.xc, densinfo)  # value: (*BD, ngrid)
    dvxc_wrt_dro = get_dvxc_wrt_dro_from_derivative_of_potinfo_wrt_ro(hamiltonian,
                                                                                derivative_of_potinfo_wrt_ro)
    return dvxc_wrt_dro

fourDtensor = four_center_elrep_tensor + \
                construct_vxc_derivative_tensor(qc._engine.hamilton, dm)
dRoothan_wrt_dC_second_term = 2 * torch.einsum("c,iakc->iakc", orboccs, fourDtensor)

density info torch.Size([52366])
started the four term integral computation for torch.Size([52366, 13])...


In [103]:
dRoothan_wrt_dC = dRoothan_wrt_dC_first_term + dRoothan_wrt_dC_second_term

#### Calculating $\frac{\partial \textbf{r}(\textbf{C};\;\vec{\theta})}{\partial \vec{\epsilon}}$

$$\frac{\partial r_{ia}(\textbf{C};\;\vec{\theta})}{\partial \epsilon_{c}} = -\delta_{ac}C_{ai}$$

In [104]:
dRoothan_wrt_depsilon = -1 * torch.einsum(
                                            "ac,ia->iac",
                                            occupied_orbitals_kronecker,
                                            coefficients
                                        )

Now, we are ready to calculate the adjoint vector.

#### Concatenate tensors:

$$\left( \frac{\partial E}{\partial \vec{X}} \right)_{N_{ao}j+c} =  \begin{cases}
   \frac{\partial E}{\partial С_{cj}} &\text{if $0 \le j < N_{orb}$} \\
   \frac{\partial E}{\partial \epsilon_{c}} &\text{if $j = N_{orb}$}
 \end{cases}$$

In [105]:
dE_wrt_dX = torch.cat((dE_wrt_dC, dE_wrt_depsilon.unsqueeze(-1)), 1) \
            .t().reshape(-1, )

$$\left( \frac{\partial \vec{Y}}{\partial \vec{X}} \right)_{N_{ao}i+a,N_{ao}j+c} =  \begin{cases}
   \frac{\partial r_{ia}}{\partial С_{cj}} &\text{if $0 \le i < N_{orb}$ and $0 \le j < N_{orb}$} \\
   \frac{\partial r_{ia}}{\partial \epsilon_{c}} &\text{if $0 \le i < N_{orb}$ and $j = N_{orb}$} \\
   \frac{\partial r_{a}}{\partial С_{cj}} &\text{if $i = N_{orb}$ and $0 \le j < N_{orb}$} \\
   \frac{\partial r_{a}}{\partial \epsilon_{c}} &\text{if $i = N_{orb}$ and $j = N_{orb}$}
 \end{cases}$$

In [106]:
N = nallorb * noccorb

upper = torch.cat((
                    dRoothan_wrt_dC.reshape((N, N)),
                    dRoothan_wrt_depsilon.reshape((N, noccorb))
                ), 1)
lower = torch.cat((
                    dnorm_wrt_dC.reshape(N, noccorb).t(),
                    dnorm_wrt_depsilon
                ), 1)
dY_wrt_dX = torch.cat((upper, lower), 0)

#### Find the adjoint vector:

In [107]:
adjoint = torch.linalg.solve(dY_wrt_dX, dE_wrt_dX)

Now, we calculate derivatives with respect to parameters:

#### Derivative from energy:
$$ \frac{\partial E[\rho](attachment:./\vec{\theta})}{\partial \vec{\theta}} = 
 \int  \frac{\partial \left(\rho(\vec{r})\epsilon_{XC}[\rho](attachment:./\vec{r};\;\vec{\theta})\right)}{\partial \vec{\theta}}d\vec{r} 
$$
so we just can put $\frac{\partial}{\partial \vec{\theta}}\left( \rho(\vec{r})\epsilon_{XC}[\rho](attachment:./\vec{r};\;\vec{\theta}) \right)$ instead of $\rho(\vec{r})\epsilon_{XC}[\rho](attachment:./\vec{r};\;\vec{\theta})$ to DQC calculation.

In [108]:
def construct_dexc_wrt_dtheta(hamiltonian, dm, parameter_number):
    densinfo = SpinParam.apply_fcn(
        lambda dm_: hamiltonian._dm2densinfo(dm_), dm)  # (spin) value: (*BD, nr)
    edens_derivative = hamiltonian.xc.get_edensityxc_derivative(densinfo, parameter_number).detach()  # (*BD, nr)
    return torch.sum(hamiltonian.grid.get_dvolume() * edens_derivative, dim=-1)

number_of_parameters = qc._engine.hamilton.xc.number_of_parameters
list_of_tensors = []
for parameter_number in range(number_of_parameters):
    current_tensor = construct_dexc_wrt_dtheta(qc._engine.hamilton, dm, parameter_number)
    list_of_tensors.append(current_tensor.unsqueeze(-1))
dE_wrt_dtheta = torch.cat(list_of_tensors, 0)

#### Derivative from normalization equations:

$$\frac{\partial r_{a}(\textbf{C})}{\partial\vec{\theta}}= 0$$

In [109]:
number_of_parameters = qc._engine.hamilton.xc.number_of_parameters
dnorm_wrt_dtheta = torch.zeros(noccorb, number_of_parameters)

#### Derivative from Roothan equations:

$$\frac{\partial r_{ia}(\textbf{C};\;\vec{\theta})}{\partial \vec{\theta}} =
\sum_j C_{aj}\int b_i(\vec{r}) \frac{\partial V_{XC}[\rho](attachment:./\vec{r};\;\vec{\theta})}{\partial \vec{\theta}} b_j(\vec{r})d\vec{r}$$

As we know, $$V_{XC}[\rho](attachment:./\vec{r};\;\vec{\theta}) = \frac{\delta E_{XC}[\rho]}{\delta\rho(\vec{r})} = \frac{\delta \left(\rho(\vec{r})\epsilon_{XC}[\rho](attachment:./\vec{r};\;\vec{\theta})\right)}{\delta\rho(\vec{r})}$$
So

$$\frac{\partial r_{ia}(\textbf{C};\;\vec{\theta})}{\partial \vec{\theta}} =
\sum_j C_{aj}\int b_i(\vec{r}) \frac{\partial}{\partial \vec{\theta}}\frac{\delta\left(\rho(\vec{r})\epsilon_{XC}[\rho](attachment:./\vec{r};\;\vec{\theta})\right)}{\delta\rho(\vec{r})} b_j(\vec{r})d\vec{r} = 
\sum_j C_{aj}\int b_i(\vec{r}) \frac{\delta}{\delta\rho(\vec{r})} \frac{\partial \left(\rho(\vec{r})\epsilon_{XC}[\rho](attachment:./\vec{r};\;\vec{\theta})\right)}{\partial \vec{\theta}} b_j(\vec{r})d\vec{r}
$$
It means, we can use $\frac{\partial}{\partial \vec{\theta}}\left( \rho(\vec{r})\epsilon_{XC}[\rho](attachment:./\vec{r};\;\vec{\theta}) \right)$ instead of $\rho(\vec{r})\epsilon_{XC}[\rho](attachment:./\vec{r};\;\vec{\theta})$ in DQC function`get_vxc()`and get suitable result. This function takes densinfo and takes functional derivative with respect to density.

In [110]:
def get_dvxc_wrt_dtheta(xc, densinfo, parameter_number):
    # mark the densinfo components as requiring grads
    with xc._enable_grad_densinfo(densinfo):
        with torch.enable_grad():
            edensity_derivative = xc.get_edensityxc_derivative(densinfo, parameter_number)  # (*BD, nr)
        grad_outputs = torch.ones_like(edensity_derivative)
        grad_enabled = torch.is_grad_enabled()

        if not isinstance(densinfo, ValGrad):  # polarized case
            raise NotImplementedError("polarized case is not implemented")
        else:  # unpolarized case
            if xc.family == 1:  # LDA
                dedn, = torch.autograd.grad(
                    edensity_derivative, densinfo.value, create_graph=grad_enabled,
                    grad_outputs=grad_outputs)

                return ValGrad(value=dedn)
            else:  # GGA and others
                raise NotImplementedError("Default dvxc wrt dro for this family is not implemented")

def contruct_dvxc_wrt_dtheta(hamiltonian, dm, parameter_number):
    densinfo = SpinParam.apply_fcn(lambda dm_: hamiltonian._dm2densinfo(dm_), dm)  # value: (*BD, nr)
    potinfo = get_dvxc_wrt_dtheta(hamiltonian.xc, densinfo, parameter_number)  # value: (*BD, nr)
    vxc_linop = hamiltonian._get_vxc_from_potinfo(potinfo)
    return vxc_linop

number_of_parameters = qc._engine.hamilton.xc.number_of_parameters
list_of_tensors = []
for parameter_number in range(number_of_parameters):
    current_tensor = contruct_dvxc_wrt_dtheta(qc._engine.hamilton, dm, parameter_number)
    list_of_tensors.append(current_tensor.fullmatrix().detach().unsqueeze(-1))
dvxc_wrt_dtheta = torch.cat(list_of_tensors, 2)

In [111]:
dRoothan_wrt_dtheta = torch.einsum("ijt,ja->iat", dvxc_wrt_dtheta, coefficients)

#### Concatenate derivatives with respect to parameters:

In [112]:
dY_wrt_dtheta = torch.cat((
                            dRoothan_wrt_dtheta.reshape(N, number_of_parameters),
                            dnorm_wrt_dtheta
                        ), 0)

#### Total derivative:

In [113]:
derivative = dE_wrt_dtheta - torch.matmul(adjoint, dY_wrt_dtheta)
derivative

tensor([ 9.9156, 19.0426], dtype=torch.float64)

#### Compare with DQC finite difference

In [114]:
fd(dqc_custom, mol_name, 1.0, 2.0)

For water:
dE/da = 9.915534184301578
dE/dp = 19.04207996972218


(tensor(9.9155, dtype=torch.float64, grad_fn=<DivBackward0>),
 tensor(19.0421, dtype=torch.float64, grad_fn=<DivBackward0>))

When calculating derivatives most of the time is spent computing calculations leading to `torch.matmul(adjoint, dY_wrt_dtheta)`, including one four-term integral. This is extremely expensive, but if we print out the result of these calculations:

In [115]:
torch.matmul(adjoint, dY_wrt_dtheta)

tensor([1.9709e-15, 2.9577e-14], dtype=torch.float64)

it could be seen that it is within margin of error from machine zero. Article shows that analytically it is zero precisely, therefore a majority of calculations could be omitted entirely.

With this, we could rewrite the code, now with PySCF, to calculate the derivatives optimally: 

### Optimal implementation with PySCF

We are only required to calculate energy partial derivatives with respect to problem parameters as described above.
It is relatively straightforward as we know the exact for of LDA functional:

In [116]:
def scf_adjoint(mol_name, a = 1.0, p = 2.0):
    mol = molecules[mol_name]
    scf_mol = gto.M(atom=mol, basis="6-31G", unit="Bohr", verbose=0)
    calc_scf = dft.RKS(scf_mol)
    calc_scf = calc_scf.define_xc_(custom_eval_xc_LDA(a, p), "LDA")
    energy = calc_scf.kernel()

    def eval_xc_wrt_a(rho):
        dexc_wrt_da = rho ** (p - 1.0)
        dvrho_wrt_da = p * rho ** (p - 1.0)
        dvxc_wrt_da = (dvrho_wrt_da, None, None, None)
        return dexc_wrt_da
    def eval_xc_wrt_p(rho):
        rho[numpy.where(rho == 0)] = numpy.finfo(float).eps # To avoid division by zero
        dexc_wrt_dp = a * numpy.log(rho) * rho ** (p - 1.0)
        dvrho_wrt_dp = a * p * numpy.log(rho) * rho ** (p - 1.0)
        dvxc_wrt_dp = (dvrho_wrt_dp, None, None, None)
        return dexc_wrt_dp

    number_of_parameters = 2 # a and p

    dE_wrt_dtheta_f = [ eval_xc_wrt_a, eval_xc_wrt_p ]

    dm_in_nonorthogonal_basis = calc_scf.make_rdm1()
    grids_coords = calc_scf.grids.coords
    grids_weights = calc_scf.grids.weights
    non_orthogonal_ao = dft.numint.eval_ao(calc_scf.mol, grids_coords, deriv=0)

    rho = dft.numint.eval_rho(calc_scf.mol, non_orthogonal_ao, 
                            dm_in_nonorthogonal_basis, xctype='LDA')
    
    list_of_dexc_wrt_dtheta = []
    for p_number in range(number_of_parameters):
        dexc_wrt_dtheta_on_grid = dE_wrt_dtheta_f[p_number](rho)

        current_dexc_wrt_dtheta = numpy.einsum('r,r,r->...', 
                                               dexc_wrt_dtheta_on_grid, 
                                               rho, 
                                               grids_weights)
        list_of_dexc_wrt_dtheta.append(current_dexc_wrt_dtheta)
    
    return list_of_dexc_wrt_dtheta

### Comparison of all derivative calculation methods

In [117]:
for mol in molecules:
    print("* DQC finite diffs:")
    start = time.time()
    ea, ep = fd(dqc_custom, mol, factor, power)
    dur = time.time() - start
    results["dqc"][mol]["ea"] = ea.item()
    results["dqc"][mol]["ep"] = ep.item()
    results["dqc"][mol]["time_d"] = dur
    print("* PySCF finite diffs:")
    start = time.time()
    ea, ep = fd(scf_custom, mol, factor, power)
    dur = time.time() - start
    results["pyscf_custom"][mol]["ea"] = ea
    results["pyscf_custom"][mol]["ep"] = ep
    results["pyscf_custom"][mol]["time_d"] = dur
    print("* DQC autograd:")
    start = time.time()
    ea, ep = dqc_autograd(mol, factor, power)
    dur = time.time() - start
    results["dqc_autograd"][mol]["ea"] = ea.item()
    results["dqc_autograd"][mol]["ep"] = ep.item()
    results["dqc_autograd"][mol]["time_d"] = dur
    print("* PySCF 'adjoint':")
    start = time.time()
    ea, ep = scf_adjoint(mol, factor, power)
    dur = time.time() - start
    results["pyscf_adjoint"][mol]["ea"] = ea
    print(f"dE/da = {ea}")
    results["pyscf_adjoint"][mol]["ep"] = ep
    results["pyscf_adjoint"][mol]["time_d"] = dur
    print(f"dE/da = {ep}")
    print("---><---")

* DQC finite diffs:
For h2:
dE/da = 0.7483576356291484
dE/dp = 1.4715006613874593
* PySCF finite diffs:
For h2:
dE/da = 0.7483589555845505
dE/dp = 1.4715019695743514
* DQC autograd:
For h2:
dE/da = 0.7483571267066685
dE/dp = 1.471532010070177
* PySCF 'adjoint':
dE/da = 0.748358476101723
dE/da = 1.4715333105617523
---><---
* DQC finite diffs:
For n2:
dE/da = 16.018380169872643
dE/dp = -17.00610112038703
* PySCF finite diffs:
For n2:
dE/da = 16.018396605818772
dE/dp = -17.006171935207703
* DQC autograd:
For n2:
dE/da = 16.018376599126125
dE/dp = -17.005452504468206
* PySCF 'adjoint':
dE/da = 16.018393025744174
dE/da = -17.005523335261252
---><---
* DQC finite diffs:
For water:
dE/da = 10.948979199902121
dE/dp = -11.080577295174976
* PySCF finite diffs:
For water:
dE/da = 10.948997128975645
dE/dp = -11.08060462691185
* DQC autograd:
For water:
dE/da = 10.948975989304921
dE/dp = -11.080098561164457
* PySCF 'adjoint':
dE/da = 10.948995156980333
dE/da = -11.08012795424238
---><---
* DQC fini

<ipython-input-86-ad976f8892da>:6: RuntimeWarning: divide by zero encountered in power
  fxc = a * p * (p - 1.0) * rho ** (p - 2.0)


## Results
In this section, the results of all computations above are assembled.

### Energy computations

In [118]:
pretty_method_names = {
    "pyscf": "PySCF (Bulitin LDA)",
    "pyscf_custom": "PySCF (Custom LDA)",
    "pyscf_ccsdt": "PySCF (CCST(T))",
    "dqc": "DQC (Custom LDA)",
    "pyscf_adjoint": "Optimal PySCF",
    "dqc_autograd": "DQC + Autograd"
}

pretty_molecule_names = {
    "h2": "H<sub>2</sub>",
    "n2": "N<sub>2</sub>",
    "water": "Water",
    "ammonia": "Ammonia"
}

In [119]:
energy_results = {}

for mol in molecules:
    energy_results[mol] = {}
    for method in results:
        if method not in [ "pyscf_adjoint", "dqc_autograd" ]:
            energy_results[mol][pretty_method_names[method]] = {
                                                            "Energy": results[method][mol]["energy"],
                                                            "Elapsed (s)": results[method][mol]["time"]
                                                        }

In [120]:
display(HTML("<h2>Energy computations results</h2>"))
for mol in molecules:
    df = pd.DataFrame(energy_results[mol])
    html = df.style.to_html()
    display(HTML(f"<h3>{pretty_molecule_names[mol]}</h3>" + html))

Energy computations results

,PySCF (Bulitin LDA),PySCF (Custom LDA),PySCF (CCST(T)),DQC (Custom LDA)
Energy,-1.038618,-1.038618,-1.151679,-1.038617
Elapsed (s),0.381758,0.355675,0.245623,0.162077


,PySCF (Bulitin LDA),PySCF (Custom LDA),PySCF (CCST(T)),DQC (Custom LDA)
Energy,-107.639474,-107.639474,-109.102647,-107.639473
Elapsed (s),0.647084,0.484608,0.642108,0.215354


,PySCF (Bulitin LDA),PySCF (Custom LDA),PySCF (CCST(T)),DQC (Custom LDA)
Energy,-75.154705,-75.154705,-76.120641,-75.154695
Elapsed (s),0.689575,0.690129,0.632575,0.300608


,PySCF (Bulitin LDA),PySCF (Custom LDA),PySCF (CCST(T)),DQC (Custom LDA)
Energy,-55.413604,-55.413604,-56.292149,-55.413601
Elapsed (s),0.760687,0.582593,0.484069,0.345989


### Derivative computations

In [121]:
derivative_results = {}

for mol in molecules:
    derivative_results[mol] = {}
    for method in results:
        if method not in [ "pyscf", "pyscf_ccsdt" ]:
            derivative_results[mol][pretty_method_names[method]] = {
                                                            "<sup>dE</sup>&frasl;<sub>da</sub>": results[method][mol]["ea"],
                                                            "<sup>dE</sup>&frasl;<sub>dp</sub>": results[method][mol]["ep"],
                                                            "Elapsed (s)": results[method][mol]["time_d"]
                                                        }

In [122]:
display(HTML("<h2>Derivatives computations results</h2>"))
for mol in molecules:
    df = pd.DataFrame(derivative_results[mol])
    html = df.style.to_html()
    display(HTML(f"<h3>{pretty_molecule_names[mol]}</h3>" + html))

Derivatives computations results

,PySCF (Custom LDA),Optimal PySCF,DQC (Custom LDA),DQC + Autograd
dE⁄da,0.748359,0.748358,0.748358,0.748357
dE⁄dp,1.471502,1.471533,1.471501,1.471532
Elapsed (s),0.945121,0.374377,0.214796,0.196723


,PySCF (Custom LDA),Optimal PySCF,DQC (Custom LDA),DQC + Autograd
dE⁄da,16.018397,16.018393,16.018380,16.018377
dE⁄dp,-17.006172,-17.005523,-17.006101,-17.005453
Elapsed (s),1.980351,0.663213,0.641580,0.326232


,PySCF (Custom LDA),Optimal PySCF,DQC (Custom LDA),DQC + Autograd
dE⁄da,10.948997,10.948995,10.948979,10.948976
dE⁄dp,-11.080605,-11.080128,-11.080577,-11.080099
Elapsed (s),2.129052,0.757619,0.683634,0.344997


,PySCF (Custom LDA),Optimal PySCF,DQC (Custom LDA),DQC + Autograd
dE⁄da,9.335451,9.335449,9.335448,9.335445
dE⁄dp,-6.248094,-6.247731,-6.248103,-6.247740
Elapsed (s),2.239646,0.835565,0.932087,0.505250


## Example of non-convergence

$N_2$ with $a = 1.0$, $p = 2.0$.

In [123]:
scf_custom('n2', 1.0, 2.0)

For n2 energy=-54.32710066141748 (a=1.0, p=2.0)


Calculation for n2 did not converge


(-54.32710066141748, False, 3.783231735229492)

In [124]:
dqc_custom('n2', 1.0, 2.0)

For n2 energy=-54.38520755799497 (a=1.0, p=2.0)


/opt/python/envs/default/lib/python3.8/site-packages/xitorch/_impls/optimize/root/rootsolver.py:163: ConvergenceWarning: The rootfinder does not converge after 50 iterations. Best |dx|=6.927e-02, |f|=3.072e-01 at iter 21
  warnings.warn(ConvergenceWarning(msg))


(tensor(-54.3852, dtype=torch.float64, grad_fn=<AddBackward0>),
 True,
 4.198075532913208)